In [ ]:
#authoer = DIego
# data = 12 december
# goal = make experiments and visulationas for report
import time
import pandas as pd
from io import StringIO
import sys

In [ ]:
import pandas as pd
from io import StringIO
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebook
import sys

def capture_game(player1, player2, time1, board):
    original_stdout = sys.stdout
    sys.stdout = StringIO()
    
    %run simulate_game.py --first={player1} --second={player2} --board=boards/{board} --time {time1}
    
    # Get the captured output as a string
    captured_output = sys.stdout.getvalue()

    # Reset redirection to the original stdout
    sys.stdout = original_stdout
    winner = captured_output[-17:-16]  # Extract the winner from the captured output
    
    return winner

def experiment1(player1, player2, amount=4):
    '''run's experiments, amount= amount of experiments for each catogry
    '''
    times = [0.1, 0.2, 1, 5] #set up for assigment
    boards = ['empty-3x3.txt', 'random-3x3.txt', 'random-4x4.txt', 'random-3x4.txt', 'random-2x3.txt', 'hard-3x3.txt', 'easy-3x3.txt']
    
    result = []
    
    for board in tqdm(boards, desc="Overall Progress", leave=False):
        for time1 in times:
            temp_result = []
            for i in tqdm(range(amount), desc=f"Board: {board}, Time: {time1}", leave=False):
                if i >= amount // 2:
                    current_player1, current_player2 = player2, player1  # Swap players
                else:
                    current_player1, current_player2 = player1, player2
                
                winner = capture_game(current_player1, current_player2, time1, board)
                temp_result.append(winner)
        
            # Store results in a DataFrame
            df = pd.DataFrame({
                'Time': [time1] * amount,
                'Winner': temp_result,
                'Board': [board] * amount,
                'Player1': [current_player1] * (amount // 2) + [current_player2] * (amount // 2),
                'Player2': [current_player2] * (amount // 2) + [current_player1] * (amount // 2)  # Fix the player columns
            })
            result.append(df)
    
    # Combine DataFrames for each board and time into a single DataFrame
    result_df = pd.concat(result, ignore_index=True)
    
    return result_df

def win_rate_table(result_table):

    # Assuming result_table is your DataFrame with simulation results
    # Replace it with the actual variable name if different
    result_table['Win'] = result_table['Winner'] == '1'  # Assuming '1' indicates Player1 won, adjust accordingly

    # Group by unique settings and calculate win rates
    win_rate_table = (
        result_table.groupby(['Time', 'Board', 'Player1', 'Player2'])['Win']
        .value_counts(normalize=True)
        .unstack(fill_value=0)
        .reset_index()
    )

    # Rename columns for clarity
    win_rate_table.columns = ['Time', 'Board', 'Player1', 'Player2', 'Loss Rate', 'Win Rate']
    return win_rate_table

In [ ]:
#expected runtime for 3x3 empty = 9*9 = 81 moves * 5 seconds, = 400 seconds, * 6 (amount) = 2400 seconds = 240 minutes = 4h
# I don't know how long the other boards take, but the empty board should take 4h.....
# each random board is half-filled, it should take ~ 2h per board
# 6 * 2 = 12 hours for random boards
# 4 hours for 3x3 empty = 16 hours
#only counting t=5
# my suggestion is doing amount = 4
# runtime = 400 * 4 ~1600 seconds for empty, ~800 seconds for non-empty
# total time = 1600 + 6 * 800 seconds = 6400 seconds = ~ 2 hours

#we could also remove random boards but idk is alot of effort aswell

result_table = experiment1(player1='team29_A2', player2='greedy_player', amount=4)  # has to be even
print(result_table)

win_rates = win_rate_table(result_table)
# Display the win rate table
print(win_rates)


In [ ]:
# Also run this for our old agent
result_table2 = experiment1(player1='team29_A2', player2='team29_A1', amount=4)
print(result_table2) #don't forget to run cell belows for our old agent
win_rates2 = win_rate_table(result_table2)
# Display the win rate table
print(win_rates2)